# Предсказание стоимости жилья


# Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import  StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

In [2]:
#инициализация spark сессии
spark = SparkSession.builder \
                    .appName("Housing") \
                    .getOrCreate()

In [3]:
df_housing = spark.read.csv('/datasets/housing.csv', inferSchema=True, header=True)

In [4]:
# вывод названий колонок 
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']).head(10))

# вывод первых 10 строк 
df_housing.show(10)

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500

In [5]:
# вывод базовых статистик
df_housing.toPandas().describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [6]:
#Проверка на наличие пропусков 
columns = df_housing.columns

for column in columns:
    print(column, df_housing.where(F.isnan(column) | F.col(column).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Исходя из первичного осмотра данных мы можем заметить пропуски в столбце `total_bedrooms`, но они составляют лишь 1% данныз так, что их можно удалить

In [7]:
df_housing = df_housing.na.drop(subset='total_bedrooms')
df_housing.describe().toPandas() 

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20433,20433,20433,20433,20433,20433,20433,20433,20433,20433
1,mean,-119.57068859198068,35.63322125972706,28.633093525179856,2636.5042333480155,537.8705525375618,1424.9469485635982,499.43346547251997,3.8711616013312273,206864.41315519012,None
2,stddev,2.003577890751096,2.1363476663779872,12.591805202182835,2185.269566977601,421.38507007403115,1133.2084897449597,382.2992258828481,1.899291249306247,115435.66709858322,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


В данных есть категориальный столбец `ocean_proximity` его нужно преобразовать, сделано это будет техникой one hot encoding

In [8]:
target = 'median_house_value' #Определение целевого показателя

categorial_col = ['ocean_proximity']

numerical_colms = ['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income']

In [9]:
indexer = StringIndexer(inputCols=categorial_col, 
                        outputCols=[c+'_idx' for c in categorial_col]) 
df_housing = indexer.fit(df_housing).transform(df_housing)

encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorial_col],
                        outputCols=[c+'_ohe' for c in categorial_col])

df_housing = encoder.fit(df_housing).transform(df_housing)
df_housing.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|        

In [10]:
#Объединение признаков в один вектор
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorial_col],
                                        outputCol="categorical_feature")
df_housing = categorical_assembler.transform(df_housing)

Числовые значения также нужно трансформировать - шкалировать значения, чтобы сильные выбросы не мешали обучению 

In [11]:
numerical_assembler = VectorAssembler(inputCols=numerical_colms, outputCol="numerical_features")

df_housing = numerical_assembler.transform(df_housing) 

In [12]:
standardScaler = StandardScaler(inputCol="numerical_features", outputCol="numerical_features_scaled")

df_housing = standardScaler.fit(df_housing).transform(df_housing) 


# Обучение моделей

Во время работы будут обучены 2 модели. 1 - со всеми признаками, 2- используя только числовые переменные, исключив категориальные. Обе модели будут обучены оценщиком(etimator) `LinearRegression`

In [13]:
features_all = ['categorical_feature','numerical_features_scaled'] #Объединение всех признаков

final_assembler = VectorAssembler(inputCols=features_all, 
                                  outputCol="features") 
df_housing = final_assembler.transform(df_housing)

df_housing.select(features_all).show(3) 


+-------------------+-------------------------+
|categorical_feature|numerical_features_scaled|
+-------------------+-------------------------+
|      (4,[3],[1.0])|     [-61.005863841998...|
|      (4,[3],[1.0])|     [-61.000872770752...|
|      (4,[3],[1.0])|     [-61.010854913244...|
+-------------------+-------------------------+
only showing top 3 rows



In [14]:
train_data, test_data = df_housing.randomSplit([.8,.2], seed=12345) #Разделение данных на обучающую и тестовую выборки
print(train_data.count(), test_data.count())

16274 4159


In [15]:
lr = LinearRegression(labelCol=target, featuresCol='features')

In [16]:
model = lr.fit(train_data) 

22/09/25 16:35:43 WARN Instrumentation: [b73c0ab8] regParam is zero, which might cause numerical instability and overfitting.
22/09/25 16:35:44 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/09/25 16:35:44 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/09/25 16:35:44 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/09/25 16:35:44 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [17]:
lr_2 = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

model_2 = lr_2.fit(train_data) 

22/09/25 16:35:47 WARN Instrumentation: [d09ca403] regParam is zero, which might cause numerical instability and overfitting.


In [18]:
predictions = model.transform(test_data)

predictions_col = 'prediction'

In [19]:
predictions2 = model_2.transform(test_data)

In [20]:
predictions.select('median_house_value','prediction').show(3)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          106700.0|213951.89157497045|
|          128900.0|203442.75691853696|
|          116100.0|229141.78939109668|
+------------------+------------------+
only showing top 3 rows



In [21]:
predictions2.select('median_house_value','prediction').show(3)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          106700.0|190472.72331598029|
|          128900.0|173844.50841793837|
|          116100.0|199496.72490793746|
+------------------+------------------+
only showing top 3 rows



# Анализ результатов

In [22]:
evaluator = RegressionEvaluator(predictionCol=predictions_col, labelCol = target) #Создание RegressionEvaluator для анализа результатов

#### Сравнение моделей метрикой RMSE (Корень средней квадратичной ошибки)

Идеальная модель будет показывать RMSE равное 0

In [23]:
print('Значение метрики RMSE для модели с категориальными признаками',
        evaluator.evaluate(predictions, {evaluator.metricName: "rmse"}))

print('Значение метрики RMSE для модели без категориальных признаков',
        evaluator.evaluate(predictions2, {evaluator.metricName: "rmse"}))

Значение метрики RMSE для модели с категориальными признаками 68127.14829705603
Значение метрики RMSE для модели без категориальных признаков 69287.75152169063


Модель с категориальными признаками показала лучший результат метрики RMSE 

#### Сравнение моделей метрикой MAE (отношение суммы всех модулей отклонений к количеству объектов)

Идеальная модель будет показывать MAE равное 0

In [24]:
print('Значение метрики MAE для модели с категориальными признаками',
        evaluator.evaluate(predictions, {evaluator.metricName: "mae"}))

print('Значение метрики MAE для модели без категориальных признаков',
        evaluator.evaluate(predictions2, {evaluator.metricName: "mae"}))

Значение метрики MAE для модели с категориальными признаками 49783.64929013866
Значение метрики MAE для модели без категориальных признаков 50979.24467565921


Модель с категориальными признаками как и в случае с первой метрикой показала лучший резльтат при измерении MAE

#### Сравнение моделей метрикой R2 

Идеальная модель будет показывать R2 равное 1

In [25]:
print('Значение метрики R2 для модели с категориальными признаками',
        evaluator.evaluate(predictions, {evaluator.metricName: "r2"}))

print('Значение метрики R2 для модели без категориальных признаков',
        evaluator.evaluate(predictions2, {evaluator.metricName: "r2"}))

Значение метрики R2 для модели с категориальными признаками 0.6479050198589585
Значение метрики R2 для модели без категориальных признаков 0.635806367136075


Модель с категориальными значениями вновь показала более точный результат

# Выводы

Модель LinearRegression обученая вместе с категориальными признаками показала более лучший результат всех метрик (RMSE, MAE, R2), по сравнению с моделью без категорильных признаков, исходя из этого можно сделать вывод, что при предсказании медианной стоимости жилья нам необходимо использовать данные близости к океану для улучшения работы модели